In [2]:
#26/8/24 Creates a K fold KNN model for each file in a folder provided they are padel

import os
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math
pd.options.mode.use_inf_as_na = True

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Padel/3D/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
finalresults = []
for df in datasets:
    df = df.dropna()
    df = df.dropna()
    df = df.drop(['Name'], axis=1)
    df = df.to_numpy()
    df = np.nan_to_num(df)
    df = pd.DataFrame(df)
    df = df.rename(columns={0: "Toxicity_Value"})
    dfarray = df.to_numpy()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        
        length = len(xtrain)
        nneigh = 100
        if length < nneigh:
            nneigh = length
        for mtry in range(1, nneigh):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = KNeighborsClassifier(n_neighbors=mtry, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)

            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = KNeighborsClassifier(n_neighbors=bestmetrics, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)
        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    finalresults.append(results[12])

/var/folders/cq/5cl8ry195dj0219k6rt6csbm0000gn/T/ipykernel_2622/4070467726.py:15: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.options.mode.use_inf_as_na = True


For dataset Hepatotoxicity_PaDEL.csv
for fold 1 test set mcc of 0.36442649807773236 valid set mcc of 0.2649573206650135
for fold 2 test set mcc of 0.32799696997113265 valid set mcc of 0.3160342302853533
for fold 3 test set mcc of 0.39352690659242356 valid set mcc of 0.3944343311190788
for fold 4 test set mcc of 0.3227167898602232 valid set mcc of 0.31577769971390995
for fold 5 test set mcc of 0.321708962392648 valid set mcc of 0.18402312981791566

validation metrics of:
positives in data 293
negatives in data 283
net accuracy = 0.6440972222222222
mcc = 0.28773398077591283
For dataset NR-PPAR-gamma_PaDEL.csv
for fold 1 test set mcc of 0.30952380952380953 valid set mcc of 0.5345224838248488
for fold 2 test set mcc of 0.25123411544939434 valid set mcc of 0.5933082268219884
for fold 3 test set mcc of 0.3779947574107829 valid set mcc of 0.3942307692307692
for fold 4 test set mcc of 0.5814571893892646 valid set mcc of 0.40121296172898085
for fold 5 test set mcc of 0.6424188694912957 valid se

For dataset Carcinogenicity_PaDEL.csv
for fold 1 test set mcc of 0.21977383072747694 valid set mcc of 0.3315938128703508
for fold 2 test set mcc of 0.22698984612511292 valid set mcc of 0.060778974111806904
for fold 3 test set mcc of 0.4428746928746929 valid set mcc of 0.14161697239501866
for fold 4 test set mcc of 0.3829268292682927 valid set mcc of 0.2360947844740331
for fold 5 test set mcc of 0.33475926709278553 valid set mcc of 0.13386318071331377

validation metrics of:
positives in data 101
negatives in data 103
net accuracy = 0.6421568627450981
mcc = 0.2844346146728079
For dataset NR-ER_PaDEL.csv
for fold 1 test set mcc of 0.4030923241028823 valid set mcc of 0.3891544543370621
for fold 2 test set mcc of 0.2592981776093564 valid set mcc of 0.3298170315194588
for fold 3 test set mcc of 0.2618700570981817 valid set mcc of 0.2767381680299383
for fold 4 test set mcc of 0.38569163012594365 valid set mcc of 0.4187583593865018
for fold 5 test set mcc of 0.4055514393921711 valid set mcc o

In [7]:
datasets[0]

,Toxicity_Value,Name,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,0,C#CC1(OC(N)=O)CCCCC1,0.0,0.0432,0.001866,36.4984,27.212309,0.0,0.0,25.0,...,0.544265,0.288326,0.379532,0.227411,0.477373,6.910318,14.149494,29.728772,0.316397,1.084316
1,0,OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,0.0,-2.9402,8.644776,38.4036,24.707102,0.0,0.0,26.0,...,0.775604,0.160624,0.549165,0.469158,0.280565,8.040570,11.914231,24.084743,0.663405,1.298889
2,1,FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F...,0.0,5.5463,30.761444,48.5205,26.599000,0.0,0.0,26.0,...,0.827758,0.093458,0.551385,0.839105,0.727467,11.990690,21.557532,44.055494,0.741637,2.117957
3,0,OCC(O)CO,0.0,-1.4081,1.982746,20.5161,13.020344,0.0,0.0,14.0,...,0.620621,0.250412,0.632803,0.524744,0.368019,3.660243,3.587085,8.230182,0.430932,1.525566
4,1,O=[N+]([O-])OCC(CO[N+](=O)[O-])O[N+](=O)[O-],0.0,2.0503,4.203730,28.5429,19.131965,0.0,0.0,20.0,...,0.772816,0.163920,0.527783,0.294689,0.402077,9.316639,16.139665,31.937263,0.659225,1.224549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,0,COc1cc(OC)c2oc3cc(OC)c(OC)c(OC4OC(OC5OCC(O)C(O...,0.0,-3.7051,13.727766,95.0445,81.689376,6.0,6.0,75.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2885,0,CC1(C)CCC2(C(=O)O)CCC3(C)C(=CCC4C5(C)CC(O)C(OC...,0.0,-2.3663,5.599376,203.4950,130.228752,0.0,0.0,123.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2886,0,N=C(N)NCCN1CCCCCCC1,0.0,-1.8499,3.422130,47.5045,36.669446,0.0,0.0,36.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2887,0,OC1C(O)C(O)C(O)C(O)C1O,0.0,-3.0642,9.389322,35.7750,23.373516,0.0,0.0,24.0,...,0.497687,0.396409,0.597116,0.564741,0.348296,5.527263,8.920058,17.975450,0.341143,1.510153


In [3]:
finalresults

[0.28773398077591283,
 0.30460012749977394,
 0.5670807236297686,
 0.3986118512736629,
 0.29711970815452493,
 0.44005060438170496,
 0.3352911848523585,
 0.251259453814803,
 0.4469870327993898,
 0.2811893561157818,
 0.28593358301364796,
 0.38277695409216156,
 0.05976143046671968,
 0.39141202231490524,
 0.2844346146728079,
 0.3513428121407796,
 0.5363984674329502,
 0.3231106243154436,
 0.22435634578416394,
 0.4571130462814951,
 0.3979630611312895,
 0.4455215914051166]

In [4]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Figure_Development/PaDEL/data/3D/padel_knn.csv')

In [14]:
resultframe

,Dataset,MCC
0,CarcinogenicityMorganfp.csv,0.300006
1,morganfp_SR-p53.csv,0.392793
2,morganfp_NR-AhR.csv,0.572339
3,morganfp_NR-ER-LBD.csv,0.453839
4,mutagenTTorsionfp.csv,0.514638
5,morganfp_NR-PPAR-gamma.csv,0.353500
6,Respiratory_ToxicityMorganfp.csv,0.421679
7,mutagenMACCSfp.csv,0.614949
8,Reproductive_ToxicityTTorsionfp.csv,0.000000
9,CarcinogenicityAvafp.csv,0.269629


In [1]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Padel/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

In [3]:
datasets[1]

,Toxicity_Value,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,0,0.0,-2.4371,5.939456,34.1807,20.039551,0.0,0.0,19.0,12.0,...,9.211803,23.026737,1.918895,15.370779,15.370779,0.000000,188.0,17.0,-0.178,58.0
22,1,0.0,4.0014,16.011202,43.4238,41.564758,12.0,12.0,27.0,21.0,...,14.957404,41.358336,1.969445,20.184704,5.072446,0.000000,860.0,38.0,3.768,110.0
33,0,0.0,0.3494,0.122080,16.7967,29.250344,11.0,11.0,25.0,17.0,...,9.522665,33.919792,1.995282,19.433820,2.414117,11.932677,537.0,23.0,0.821,86.0
66,0,0.0,-0.8075,0.652056,39.6471,49.714102,12.0,12.0,41.0,27.0,...,10.269214,54.018524,2.000686,32.395517,9.793396,8.638308,1752.0,46.0,3.480,150.0
90,1,0.0,-0.2952,0.087143,64.8328,48.247481,6.0,6.0,41.0,24.0,...,8.587106,48.337933,2.014081,19.320203,16.736492,0.000000,1137.0,50.0,0.911,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6269,1,0.0,-0.7769,0.603574,15.9147,37.474309,12.0,12.0,31.0,18.0,...,7.776983,36.143595,2.007977,7.893785,7.893785,0.000000,650.0,24.0,4.147,88.0
6270,0,0.0,-0.6722,0.451853,61.3223,63.497446,12.0,12.0,51.0,29.0,...,8.394099,58.471448,2.016257,19.524183,9.950784,3.455758,2507.0,42.0,5.272,146.0
6282,1,0.0,-3.4627,11.990291,185.1441,128.750717,0.0,0.0,122.0,53.0,...,6.143311,106.097356,2.001837,30.578223,30.578223,0.000000,11875.0,107.0,4.629,292.0
6303,0,0.0,-5.2280,27.331984,71.3748,68.005067,12.0,12.0,55.0,36.0,...,9.655717,72.615360,2.017093,38.538093,19.623783,12.495163,4309.0,60.0,0.945,194.0
